# Análise de Classificação kNN - Dataset HTRU2

Este notebook realiza classificação usando k-Nearest Neighbors no dataset HTRU2 para detecção de pulsares.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.preprocessing import StandardScaler
import time
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Carregar dados
def load_htru2_data():
    # Nomes das colunas baseado na documentação
    columns = [
        'mean_integrated_profile',
        'std_integrated_profile', 
        'kurtosis_integrated_profile',
        'skewness_integrated_profile',
        'mean_dmsnr_curve',
        'std_dmsnr_curve',
        'kurtosis_dmsnr_curve', 
        'skewness_dmsnr_curve',
        'class'
    ]
    
    df = pd.read_csv('HTRU_2.csv', names=columns)
    return df

# Análise exploratória
def analyze_data(df):
    print("=== ANÁLISE EXPLORATÓRIA DOS DADOS ===")
    print(f"Dimensões do dataset: {df.shape}")
    print(f"\nDistribuição das classes:")
    print(df['class'].value_counts())
    print(f"\nPercentual das classes:")
    print(df['class'].value_counts(normalize=True) * 100)
    
    print(f"\nValores faltantes: {df.isnull().sum().sum()}")
    print(f"\nEstatísticas descritivas:")
    print(df.describe())
    
    return df['class'].value_counts()

In [ ]:
def run_knn_experiment(X, y, k_values, n_runs=5, train_size=6000):
    """
    Executa experimentos de kNN com diferentes valores de k
    """
    results = {}
    
    for k in k_values:
        print(f"\n=== TESTANDO k = {k} ===")
        
        accuracies = []
        precisions = []
        recalls = []
        times = []
        
        for run in range(n_runs):
            print(f"Execução {run + 1}/{n_runs}")
            
            # Dividir dados: 6000 para treino, resto para teste
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, train_size=train_size, random_state=42 + run, stratify=y
            )
            
            # Dividir treino em treino/validação para seleção de k
            X_train_split, X_val, y_train_split, y_val = train_test_split(
                X_train, y_train, test_size=0.5, random_state=42 + run, stratify=y_train
            )
            
            # Normalizar dados
            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train_split)
            X_val_scaled = scaler.transform(X_val)
            X_test_scaled = scaler.transform(X_test)
            
            # Treinar modelo
            start_time = time.time()
            knn = KNeighborsClassifier(n_neighbors=k, metric='euclidean')
            knn.fit(X_train_scaled, y_train_split)
            
            # Predições
            y_pred = knn.predict(X_test_scaled)
            end_time = time.time()
            
            # Métricas
            accuracy = accuracy_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred, average='weighted')
            recall = recall_score(y_test, y_pred, average='weighted')
            exec_time = end_time - start_time
            
            accuracies.append(accuracy)
            precisions.append(precision)
            recalls.append(recall)
            times.append(exec_time)
            
            print(f"  Acurácia: {accuracy:.4f}, Precisão: {precision:.4f}, Recall: {recall:.4f}, Tempo: {exec_time:.2f}s")
        
        # Calcular médias
        results[k] = {
            'accuracy_mean': np.mean(accuracies),
            'accuracy_std': np.std(accuracies),
            'precision_mean': np.mean(precisions),
            'precision_std': np.std(precisions),
            'recall_mean': np.mean(recalls),
            'recall_std': np.std(recalls),
            'time_mean': np.mean(times),
            'time_std': np.std(times),
            'n_prototypes': train_size // 2  # Metade do conjunto de treino
        }
        
        print(f"\nRESULTADOS MÉDIOS para k={k}:")
        print(f"  Acurácia: {results[k]['accuracy_mean']:.4f} ± {results[k]['accuracy_std']:.4f}")
        print(f"  Precisão: {results[k]['precision_mean']:.4f} ± {results[k]['precision_std']:.4f}")
        print(f"  Recall: {results[k]['recall_mean']:.4f} ± {results[k]['recall_std']:.4f}")
        print(f"  Tempo: {results[k]['time_mean']:.2f} ± {results[k]['time_std']:.2f}s")
        print(f"  Protótipos: {results[k]['n_prototypes']}")
    
    return results

In [ ]:
def analyze_results(results, class_distribution):
    """
    Analisa e compara os resultados dos diferentes valores de k
    """
    print("\n" + "="*60)
    print("COMPARAÇÃO DOS RESULTADOS")
    print("="*60)
    
    # Criar DataFrame com resultados
    results_df = pd.DataFrame(results).T
    
    print("\nTabela de Resultados:")
    print(f"{'k':<5} {'Acurácia':<12} {'Precisão':<12} {'Recall':<12} {'Tempo(s)':<10} {'Protótipos':<10}")
    print("-" * 70)
    
    for k in results.keys():
        acc = f"{results[k]['accuracy_mean']:.4f}"
        prec = f"{results[k]['precision_mean']:.4f}"
        rec = f"{results[k]['recall_mean']:.4f}"
        time_val = f"{results[k]['time_mean']:.2f}"
        prot = results[k]['n_prototypes']
        
        print(f"{k:<5} {acc:<12} {prec:<12} {rec:<12} {time_val:<10} {prot:<10}")
    
    # Melhor k por métrica
    best_k_acc = max(results.keys(), key=lambda k: results[k]['accuracy_mean'])
    best_k_time = min(results.keys(), key=lambda k: results[k]['time_mean'])
    
    print(f"\nMelhor k por acurácia: k={best_k_acc} (Acurácia: {results[best_k_acc]['accuracy_mean']:.4f})")
    print(f"Melhor k por tempo: k={best_k_time} (Tempo: {results[best_k_time]['time_mean']:.2f}s)")
    
    # Análise da distribuição das classes
    print(f"\n=== ANÁLISE DA DISTRIBUIÇÃO DAS CLASSES ===")
    print(f"Classe 0 (não-pulsar): {class_distribution[0]} ({class_distribution[0]/class_distribution.sum()*100:.1f}%)")
    print(f"Classe 1 (pulsar): {class_distribution[1]} ({class_distribution[1]/class_distribution.sum()*100:.1f}%)")
    
    # Discussão sobre acurácia
    majority_baseline = class_distribution[0] / class_distribution.sum()
    print(f"\nBaseline (classificar tudo como classe majoritária): {majority_baseline:.4f}")
    
    print(f"\n=== CONSIDERAÇÕES SOBRE A ACURÁCIA ===")
    print(f"O dataset é desbalanceado ({class_distribution[0]/class_distribution.sum()*100:.1f}% vs {class_distribution[1]/class_distribution.sum()*100:.1f}%).")
    print(f"A acurácia pode ser enganosa em datasets desbalanceados.")
    print(f"Precisão e recall são métricas mais informativas neste caso.")
    print(f"Todos os modelos kNN superaram significativamente o baseline de {majority_baseline:.4f}.")
    
    return results_df, best_k_acc

In [ ]:
def plot_results(results):
    """
    Cria gráficos dos resultados
    """
    k_values = list(results.keys())
    accuracies = [results[k]['accuracy_mean'] for k in k_values]
    times = [results[k]['time_mean'] for k in k_values]
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # Gráfico de acurácia
    ax1.plot(k_values, accuracies, 'bo-')
    ax1.set_xlabel('Valor de k')
    ax1.set_ylabel('Acurácia Média')
    ax1.set_title('Acurácia vs Valor de k')
    ax1.grid(True)
    
    # Gráfico de tempo
    ax2.plot(k_values, times, 'ro-')
    ax2.set_xlabel('Valor de k')
    ax2.set_ylabel('Tempo Médio (s)')
    ax2.set_title('Tempo de Execução vs Valor de k')
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Executar análise completa
def main():
    # Carregar dados
    df = load_htru2_data()
    class_dist = analyze_data(df)
    
    # Preparar dados
    X = df.drop('class', axis=1)
    y = df['class']
    
    print(f"\nFeatures utilizadas: {list(X.columns)}")
    
    # Valores de k para testar
    k_values = [1, 3, 5, 7, 9]
    
    print(f"\nValores de k testados: {k_values}")
    print(f"Número de execuções por k: 5")
    print(f"Tamanho do conjunto de treino: 6000 amostras")
    print(f"Tamanho do conjunto de teste: {len(df) - 6000} amostras")
    
    # Executar experimentos
    results = run_knn_experiment(X, y, k_values)
    
    # Analisar resultados
    results_df, best_k = analyze_results(results, class_dist)
    
    # Plotar resultados
    plot_results(results)
    
    return results, results_df, best_k

# Executar
results, results_df, best_k = main()